In [1]:
from typing import Union, Callable

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
from googletrans import Translator

translator = Translator()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/home/migueldcarvalho/miniforge3/envs/DataScience/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


In [30]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device='cpu')
client = QdrantClient(path="../metadados/VectorStore/") # Carrega vectorstore em disco

## tool class

In [80]:
class Tool:
    """
    A class representing a reusable piece of code (Tool).

    Attributes:
        name (str): Name of the tool.
        description (str): A textual description of what the tool does.
        func (callable): The function this tool wraps.
        arguments (list): A list of arguments.
        outputs (str or list): The return type(s) of the wrapped function.
    """
    def __init__(self,
                 name: str,
                 description: str,
                 func: Callable,
                 arguments: list,
                 outputs: str):
        self.name = name
        self.description = description
        self.func = func
        self.arguments = arguments
        self.outputs = outputs

    def to_string(self) -> str:
        """
        Return a string representation of the tool,
        including its name, description, arguments, and outputs.
        """
        args_str = ", ".join([
            f"{arg_name}: {arg_type}" for arg_name, arg_type in self.arguments
        ])

        return (
            f"Tool Name: {self.name},"
            f" Description: {self.description},"
            f" Arguments: {args_str},"
            f" Outputs: {self.outputs}"
        )

    def __call__(self, *args, **kwargs):
        """
        Invoke the underlying function (callable) with provided arguments.
        """
        return self.func(*args, **kwargs)

## Chatbot class

In [ ]:
import json

class Chatbot:

    def __init__(self, **kwargs):

        self.tokenizer = kwargs.get('tokenizer')
        self.model = kwargs.get('model')
        self.history = []

    def agent(self, prompt: dict) -> str:

        # Suas ferramentas bem descritas
        tools_description = [
            {
                "name": "open_data_search",
                "description": "Busca por dados públicos aberto governamentais. "
                "Exemplos:"
                "Quero dados sobre..."
                ""
                "Input: query de busca (string).",
                "parameters": { "type": "object", "properties": { "query": {"type": "string"} } }
            },
            {
                "name": "soma",
                "description": "Deve ser acionado quando necessário somar dois numeros, Calculadora de adição (A + B). Input: a (float), b (float).",
                "parameters": { "type": "object", "properties": { "a": {"type": "number"}, "b": {"type": "number"} } }
            }
        ]

        tools_str = json.dumps(tools_description, indent=2, ensure_ascii=False)

        system_instruction = f"""
        Você é um assistente AI útil e preciso.

        ### FERRAMENTAS DISPONÍVEIS
        Você tem acesso às seguintes ferramentas para responder ao usuário:

        {tools_str}

        ### INSTRUÇÕES DE FORMATO
        Para usar uma ferramenta, você DEVE responder APENAS com um bloco JSON no seguinte formato, e nada mais:

        {{
            "tool": "nome_da_ferramenta",
            "parameters": {{
                "parametro1": "valor",
                "parametro2": "valor"
            }}
        }}

        Se nenhuma ferramenta for necessária, responda diretamente ao usuário.
        """
        messages = [
            {'role' : 'system', 'content': system_instruction},
            {'role' : 'user', 'content' : prompt}
        ]

        text = self.tokenizer.apply_chat_template( # Formata 
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
        )
        print(text)
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)

        # conduct text completion
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=32768
        )
        output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

        # parsing thinking content
        try:
            # rindex finding 151668 (</think>)
            index = len(output_ids) - output_ids[::-1].index(151668)
        except ValueError:
            index = 0

        thinking_content = self.tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
        content = self.tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

        print("thinking content:", thinking_content)
        print("content:", content)
        return content
    
    async def open_data_search(self, query: str, client, encoder) -> dict:

        #encoder = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device='cpu')
        translator = Translator()
        #client = QdrantClient(path="../metadados/VectorStore/") # Carrega vectorstore em disco

        task = "Você é um motor de busca, devolva dados mais relevantes com base na busca"

        query_pt = f"Instrução: {task} Query: {query}"
        query_en = await translator.translate(query_pt, src='pt', dest='en')

        query = query_en.text

        print("Query em português:", query_pt)
        print("Query utilizada:", query)

        hits = self.client.query_points(
            collection_name="Catalogo_metadados",
            query=self.encoder.encode(query).tolist(),
            limit=20,
        ).points

        print(hits)

        for hit in hits:
            print("score:", hit.score)
            print("Titulo: ",hit.payload.get('title'))
            print("Nome: ", hit.payload.get('nome'))
            print("Descrição: ", hit.payload.get('descricao'))
            print('\n')

    def chat(self, query: str, client, encoder) -> str:
        #tool = eval(self.agent(query)).get('tool')
        #print(tool)
        await self.open_data_search(query, client, encoder)
        #self,eval(tool)(query, client, encoder)

    def clean_history(self):
        self.history.clear()


In [56]:
chatbot = Chatbot(model=model, tokenizer=tokenizer)

query = "Quero dados sobre infecção hospitalar"

tool = chatbot.chat(query, client, encoder)

/tmp/ipykernel_7028/527675232.py:123: RuntimeWarning: coroutine 'Chatbot.open_data_search' was never awaited
  self.open_data_search(query, client, encoder)


NameError: name 'open_data_search' is not defined

In [ ]:
@Tool
def soma(a:int, b:int) -> Union[int, float]:
    """
    Docstring for soma

    Soma dois numeros de tipo int e/ou float
    args:
        a: int/float
        b: int/float
    Returns:
        Retorna a soma a+b : int/float
    """
    return a + b